In [3]:
import tequila as tq
from utility import *
import copy
P_X = np.array([[0, 1], [1, 0]])
P_Y = np.array([[0, 0 -1j],[0+1j,0]])
P_Z = np.array([[1, 0], [0, -1]])

In [4]:
def str2mat(hamiltonian_string,n_qubits):
    total_hamiltonian = None  #stores the total hamiltonian
    first_flag = True
    
    for line in hamiltonian_string.splitlines():
        temp_hamiltonian = None #stores the temporary hamiltonian
        matrix_operand = None
        var_start = line.find('[')
        var_end = line.find(']')
        coeff = float(line[0:var_start]) #coefficients for individual parts
        temp_ham_set = line[var_start+1:var_end].split() #variables for individual parts
        
        indiv_ham_set = ['I' for i in range(0,n_qubits)] #this decides what operator affects which qubit, initialize to identity
        
        if len(temp_ham_set)!=0:
            for operator in temp_ham_set:
                q_index = int(operator[1:len(str(n_qubits))+1]) #Find out which qubit it acts on
                if operator[0]=='X':
                    indiv_ham_set[q_index] = 'P_X'
                elif operator[0]=='Y':
                    indiv_ham_set[q_index] = 'P_Y'
                elif operator[0]=='Z':
                    indiv_ham_set[q_index] = 'P_Z'
        print(temp_ham_set)
        print(indiv_ham_set)
        
        for i in range(0,n_qubits-1):
            if i== 0:
                #reinitialize temp_hamiltonian from first qubit
                if indiv_ham_set[i] == 'I':
                    temp_hamiltonian = copy.deepcopy(np.eye(2))
                elif indiv_ham_set[i] == 'P_X':
                    temp_hamiltonian = copy.deepcopy(P_X)
                elif indiv_ham_set[i] == 'P_Y':
                    temp_hamiltonian = copy.deepcopy(P_Y)
                elif indiv_ham_set[i] == 'P_Z':
                    temp_hamiltonian = copy.deepcopy(P_Z)
            #now for the operand to which the temp_hamiltonian will be kroneckered
            if indiv_ham_set[i+1] == 'I':
                matrix_operand = copy.deepcopy(np.eye(2))
            elif indiv_ham_set[i+1] == 'P_X':
                matrix_operand = copy.deepcopy(P_X)
            elif indiv_ham_set[i+1] == 'P_Y':
                matrix_operand = copy.deepcopy(P_Y)
            elif indiv_ham_set[i+1] == 'P_Z':
                matrix_operand = copy.deepcopy(P_Z)
                
            #Kronecker together
            temp_hamiltonian = np.kron(temp_hamiltonian,matrix_operand)
        
        #print(coeff*temp_hamiltonian)
        #print(temp_hamiltonian.shape)
        #print(temp_hamiltonian[0,:])
        #print(np.count_nonzero(temp_hamiltonian[0,:]))
        if first_flag == True:
            temp_hamiltonian = coeff*temp_hamiltonian
            total_hamiltonian = copy.deepcopy(temp_hamiltonian)
            first_flag = False
            #print(total_hamiltonian)
        else:
            total_hamiltonian = total_hamiltonian + coeff*temp_hamiltonian
    print(total_hamiltonian)

In [5]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(h2)


#Get additional information from tequila
xyz_data = get_molecular_data('h2', geometry=2.5, xyz_format=True)
basis='sto-3g'
h2_tq = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)


-0.32760818967480887 [] +
-0.04919764587136755 [X0 X1 Y2 Y3] +
0.04919764587136755 [X0 Y1 Y2 X3] +
0.04919764587136755 [Y0 X1 X2 Y3] +
-0.04919764587136755 [Y0 Y1 X2 X3] +
0.13716572937099497 [Z0] +
0.15660062488237947 [Z0 Z1] +
0.10622904490856075 [Z0 Z2] +
0.15542669077992832 [Z0 Z3] +
0.13716572937099492 [Z1] +
0.15542669077992832 [Z1 Z2] +
0.10622904490856075 [Z1 Z3] +
-0.13036292057109117 [Z2] +
0.16326768673564346 [Z2 Z3] +
-0.13036292057109117 [Z3]


In [6]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h2, n_spin_orbitals=2*h2_tq.n_orbitals, n_electrons=h2_tq.n_electrons, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -0.5310513494337641 [] +
0.1967905834854702 [X0] +
-0.5350572998841723 [Z0]


In [8]:
h2_small_str = format(taper_hamiltonian(h2, n_spin_orbitals=2*h2_tq.n_orbitals, n_electrons=h2_tq.n_electrons, qubit_transf=qubit_transf))